In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Tạo folder data để các bạn upload file 3 file data được tạo ra ở step 0 vào folder data
# Các bạn upload file hand_gesture.yaml ngoài folder data
!mkdir data/

In [ ]:
# Nếu đã chạy thành công step 0 thì các bạn không chạy cell này và chỉ upload 3 file data của các bạn vào thư mục data
# Và upload file hand_gesture.yaml bên ngoài thư mục data
# Nếu không chạy được ở step 0 thì các bạn run cell này để download data
!gdown  1gzWOtABiVmJ38usCSDe5F9gR2tECt3zu -O data/
!gdown  15lwipssmC_K82ukRfb0uVCiDH1TZ3QCf -O data/
!gdown  1nIo1_wBmkovz-u_BCsV5c1Kbz6ZqoKwq -O data/
# Download file hand_gesture.yam
!gdown  1ZteHYSgbuZu_GcUJHW8ZzoZv1DE8-oLw

In [1]:
!pip install mediapipe==0.10.18

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 14.5 MB/s eta 0:00:00


In [2]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 17.3 MB/s eta 0:00:00


In [12]:
import os
import cv2
import yaml
import torch
import numpy as np
import pandas as pd
from  torch import nn
import mediapipe as mp
from torch import optim
from datetime import datetime
from torchmetrics import Accuracy
from torch.utils.data import Dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        list_label = label_dict_from_config_file("/content/drive/MyDrive/AI VIETNAM/Module 5/Hint/step0_generate_data/hand_gesture.yaml")

        self.linear_relu_stack = nn.Sequential(
            ################## Your Code Here ################## Q1
            # '''Hoàn thành đoạn code để xây dựng một model gồm có 4 hidden layer,
            # lần lượng input và output là (63, 128), (128, 128), (128, 128), (128, 128).
            # Layer đầu tiên được theo sau bổi một Relu và Batchnorm1d.
            # Layer thứ 2, 3, và 4 được theo sau bỏi Relu và Dropout với rate lần lượt là 0.4, 0.4, 0.6.
            # Output layer có nhịêm vụ phân loại với input là 128 và output là số lượng class cử chỉ
            # '''

            nn.Linear(63, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.Linear(128, len(list_label)),
            ####################################################

        )
    def forward(self, x):
        ################## Your Code Here ################## Q2
        # ''' Hoàn thành code để thực hiện forward dự đoán cử chỉ với input x.
        # Thực hiệnt flatten x
        # Pass x vừa flatten vào linear_relu_stack
        # Return  logits (outputs từ layer cuối cùng)
        # '''


        x = self.flatten(x)
        logits = self.linear_relu_stack(x)

        return logits
        ####################################################


    def predict(self,x,threshold=0.8):
        logits = self(x)
        softmax_prob = nn.Softmax(dim=1)(logits)
        chosen_ind = torch.argmax(softmax_prob,dim=1)
        return torch.where(softmax_prob[0,chosen_ind]>threshold,chosen_ind,-1)

    def predict_with_known_class(self,x):
        logits = self(x)
        softmax_prob = nn.Softmax(dim=1)(logits)
        return torch.argmax(softmax_prob,dim=1)

    def score(self,logits):
        return -torch.amax(logits,dim=1)

In [7]:
def label_dict_from_config_file(relative_path):
    with open(relative_path,"r") as f:
       label_tag = yaml.full_load(f)["gestures"]
    return label_tag

In [8]:
class HandLandmarksDetector():
    def __init__(self) -> None:
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        self.mp_hands = mp.solutions.hands
        self.detector = self.mp_hands.Hands(False,max_num_hands=1,min_detection_confidence=0.5)

    def detectHand(self,frame):
        hands = []
        frame = cv2.flip(frame, 1)
        annotated_image = frame.copy()
        results = self.detector.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if results.multi_hand_landmarks is not None:
            for hand_landmarks in results.multi_hand_landmarks:
                hand = []
                self.mp_drawing.draw_landmarks(
                    annotated_image,
                    hand_landmarks,
                    self.mp_hands.HAND_CONNECTIONS,
                    self.mp_drawing_styles.get_default_hand_landmarks_style(),
                    self.mp_drawing_styles.get_default_hand_connections_style())
                for landmark in hand_landmarks.landmark:
                    x,y,z = landmark.x,landmark.y,landmark.z
                    hand.extend([x,y,z])
            hands.append(hand)
        return hands,annotated_image

In [9]:
class CustomImageDataset(Dataset):
    def __init__(self, data_file):
        self.data = pd.read_csv(data_file)
        self.labels = torch.from_numpy(self.data.iloc[:,0].to_numpy())

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        one_hot_label = self.labels[idx]
        torch_data = torch.from_numpy(self.data.iloc[idx,1:].to_numpy(dtype=np.float32))
        return torch_data, one_hot_label

In [35]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.watched_metrics = np.inf
        self.best_metric = np.inf

    def early_stop(self, current_value):
        if current_value < self.watched_metrics:
            self.watched_metrics = current_value
            self.best_metric = current_value
            self.counter = 0
        elif current_value > (self.watched_metrics + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False


In [36]:
def train(trainloader, val_loader, model, loss_function, early_stopper, optimizer):
    best_vloss = 1_000_000
    timestamp = datetime.now().strftime('%d-%m %H:%M')
    model = model.to(device)

    for epoch in range(300):
        # Training step
        model.train(True)
        running_loss = 0.0
        acc_train = Accuracy(num_classes=len(LIST_LABEL), task='MULTICLASS').to(device)

        for batch_number, data in enumerate(trainloader):
            inputs, labels = data
            inputs = inputs.to(torch.float32).to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            preds = model(inputs)

            loss = loss_function(preds, labels)
            loss.backward()
            optimizer.step()

            acc_train.update(model.predict_with_known_class(inputs), labels)
            running_loss += loss.item()

        avg_loss = running_loss / len(trainloader)

        # Validation step
        model.train(False)
        running_vloss = 0.0
        acc_val = Accuracy(num_classes=len(LIST_LABEL), task='MULTICLASS').to(device)

        for i, vdata in enumerate(val_loader):
            vinputs, vlabels = vdata
            vinputs = vinputs.to(torch.float32).to(device)
            vlabels = vlabels.to(device)

            preds = model(vinputs)
            vloss = loss_function(preds, vlabels)
            running_vloss += vloss.item()

            acc_val.update(model.predict_with_known_class(vinputs), vlabels)

        # Log metrics
        print(f"Epoch {epoch}:")
        print(f"Accuracy train: {acc_train.compute().item()}, val: {acc_val.compute().item()}")
        avg_vloss = running_vloss / len(val_loader)
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

        # Save the best model
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss

            # Sửa đường dẫn
            best_model_path = os.path.join(save_path, f'model_{timestamp}_{model.__class__.__name__}_best')
            torch.save(model.state_dict(), best_model_path)

        # Early stopping
        if early_stopper.early_stop(avg_vloss):
            print(f"Early stopping at epoch {epoch}. Minimum validation loss observed: {early_stopper.best_metric:.4f}")
            break

    # Lưu mô hình cuối cùng
    model_path = os.path.join(save_path, f'model_{timestamp}_{model.__class__.__name__}_last')
    torch.save(model.state_dict(), model_path)

    print(f"Final validation accuracy: {acc_val.compute().item()}")
    return model, best_model_path


In [37]:
DATA_FOLDER_PATH="/content/drive/MyDrive/AI VIETNAM/Module 5/Hint/data"
LIST_LABEL = label_dict_from_config_file("/content/drive/MyDrive/AI VIETNAM/Module 5/Hint/step0_generate_data/hand_gesture.yaml")
train_path = os.path.join(DATA_FOLDER_PATH, "/content/drive/MyDrive/AI VIETNAM/Module 5/Hint/data/landmark_train.csv")
val_path = os.path.join(DATA_FOLDER_PATH, "/content/drive/MyDrive/AI VIETNAM/Module 5/Hint/data/landmark_val.csv")
save_path = '/content/drive/MyDrive/AI VIETNAM/Module 5/Hint/models'
os.makedirs(save_path,exist_ok=True)

trainset = CustomImageDataset(train_path)
################## Your Code Here ################## Q3
    # '''Hoàn thành code để thực hiện khởi tạo DataLoader cho trainset với
    # batch_size 40 và cho phép xáo trộn
    # '''
trainloader = torch.utils.data.DataLoader(trainset,batch_size=40, shuffle=True)
####################################################

valset = CustomImageDataset(os.path.join(val_path))
val_loader = torch.utils.data.DataLoader(valset,batch_size=50, shuffle=False)

################## Your Code Here ################## Q8
'''Hoàn thành code để thực hiện khởi tạo NeuralNetwork model đã xây dựng ở trên,
khởi tạo hàm loss sử dụng CrossEntropyLoss và khởi tạo early stopper với patience
là 30 và min_delta là 0.01
'''
model = NeuralNetwork()
model = model.to(device)
loss_function = nn.CrossEntropyLoss()
early_stopper = EarlyStopper(patience=30, min_delta=0.01)
####################################################

################## Your Code Here ################## Q4
'''Hoàn thành code để thực hiện cấu hình Adam optimizer cho các tham số của
model với tốc độ học là 0.0001
'''
optimizer = optim.Adam(model.parameters(), lr=0.0001)
####################################################


model, best_model_path = train(trainloader, val_loader, model, loss_function, early_stopper, optimizer)

Epoch 0:
Accuracy train: 0.1952894926071167, val: 0.16723549365997314
LOSS train 1.6086809772711534 valid 1.6045911312103271
Epoch 1:
Accuracy train: 0.2688910663127899, val: 0.34812286496162415
LOSS train 1.5922724467057447 valid 1.5858725905418396
Epoch 2:
Accuracy train: 0.33856722712516785, val: 0.40273037552833557
LOSS train 1.5762364039054284 valid 1.5698782205581665
Epoch 3:
Accuracy train: 0.41118744015693665, val: 0.37542662024497986
LOSS train 1.5494236533458416 valid 1.538580556710561
Epoch 4:
Accuracy train: 0.4631992280483246, val: 0.3686006963253021
LOSS train 1.5076599442041838 valid 1.488886038462321
Epoch 5:
Accuracy train: 0.4867517054080963, val: 0.3788395822048187
LOSS train 1.4455300798782935 valid 1.4046536087989807
Epoch 6:
Accuracy train: 0.499509334564209, val: 0.3890784978866577
LOSS train 1.3333112047268794 valid 1.2897402445475261
Epoch 7:
Accuracy train: 0.5230618119239807, val: 0.43344709277153015
LOSS train 1.2170250713825226 valid 1.1743092040220897
Epoc

In [60]:
list_label = label_dict_from_config_file("/content/drive/MyDrive/AI VIETNAM/Module 5/Hint/step0_generate_data/hand_gesture.yaml")
DATA_FOLDER_PATH="/content/drive/MyDrive/AI VIETNAM/Module 5/Hint/data"
testset = CustomImageDataset(os.path.join(DATA_FOLDER_PATH, "/content/drive/MyDrive/AI VIETNAM/Module 5/Hint/data/landmark_test.csv"))

# Test DataLoader instantiation
################## Your Code Here ################## Q6
''' Hoàn thành code bên dưới để  khởi tạo DataLoader cho testset with batch size
20, không cho phép shuffle
'''
test_loader = torch.utils.data.DataLoader(testset, batch_size=20, shuffle=False)
####################################################



network = NeuralNetwork()
network.load_state_dict(torch.load(best_model_path, weights_only=False))

network.eval()
acc_test = Accuracy(num_classes=len(list_label), task='MULTICLASS')
for i, test_data in enumerate(test_loader):
    test_input, test_label = test_data
    ################## Your Code Here ################## Q7
    # '''Hoàn thành code bên dưới để  predict class của cử chỉ và update accuracy
    # với kết quả predict và true labels
    # '''
    preds = network(test_input)
    acc_test.update(network.predict_with_known_class(test_input).cpu(), test_label)


    ####################################################

print(network.__class__.__name__)
print(f"Accuracy of model:{acc_test.compute().item()}")
print("========================================================================")

NeuralNetwork
Accuracy of model:0.9763779640197754
